In [4]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import cv2
from keras.utils import np_utils
from tensorflow.keras import datasets, layers, models,Model
from keras.callbacks import ModelCheckpoint,EarlyStopping
from multiprocessing import Pool
import matplotlib.pyplot as plt

In [5]:
def load_samples():
    data = pd.read_csv("csv\sampled\ct_train_final.csv",index_col=0)  
    c=0
    for row in data.itertuples(index=False):
      samples.append([x for x in row])
      c+=1
      # if(c==): # to break when required sample is already pulled
      #   break
    return samples

In [ ]:
samples=[]
samples=load_samples()
print (samples[0:5])
print(len(samples))

In [7]:
def create_model():
  model=models.Sequential()
  model.add(layers.Conv2D(16,kernel_size=(3, 3), activation='relu', input_shape=(275,275,1)))
  model.add(layers.BatchNormalization())
  model.add(layers.AveragePooling2D(pool_size=(2, 2)))

  # normliztion to use overfitting use drop out only in full connected network ie dense layer
  
  model.add(layers.BatchNormalization())

  model.add(layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.AveragePooling2D(pool_size=(2, 2)))

  model.add(layers.Conv2D(128, kernel_size=(3, 3), activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.AveragePooling2D(pool_size=(2, 2)))

  model.add(layers.BatchNormalization())

  model.add(layers.Flatten())
  model.add(layers.Dense(128, activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.Dropout(0.3))

  model.add(layers.Dense(16, activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.Dropout(0.3))

  model.add(layers.Dense(5,activation='sigmoid'))
  return model


In [8]:
bone_model=create_model()
brain_model=create_model()
blood_model=create_model()

In [9]:
combined=layers.concatenate([bone_model.output,brain_model.output,blood_model.output])
combined_model = layers.Dense(15, activation="relu")(combined)
combined_model = layers.Dense(5, activation="sigmoid")(combined_model)

In [10]:
model = Model(inputs=[bone_model.input, brain_model.input,blood_model.input], outputs=combined_model)

In [ ]:
model.summary()

In [ ]:
label.shape[0]

In [14]:
#Changing file directory and patience set to 5 & saving only best models
filepath="models\weights-improvement-{epoch:02d}-{accuracy:02f}.hdf5"
earlystop=EarlyStopping(monitor='accuracy', mode='max',  patience=5)
checkpoint = ModelCheckpoint(filepath, monitor='accuracy', save_best_only=True, mode='max')
callbacks_list = [earlystop,checkpoint]

In [12]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
#Use this generator functions
def generator(samples, batch_size=32):
    """
    Yields the next training batch.
    Suppose `samples` is an array [[image1_filename,label1], [image2_filename,label2],...].
    """
    num_samples = len(samples)
    while True: # Loop forever so the generator never terminates
        shuffle(samples)
 
        # Get index to start each batch: [0, batch_size, 2*batch_size, ..., max multiple of batch_size &lt;= num_samples]
        for offset in range(0, num_samples, batch_size):
            # Get the samples you'll use in this batch
            batch_samples = samples[offset:offset+batch_size]
 
            # Initialise X_train and y_train arrays for this batch
            X_train = []
            X_trains=[]
            X_trainss=[]
            y_train = []
            
            # For each example
            for batch_sample in batch_samples:
                # Load image (X)
                filename = 'data\\processed\\bone_window\\'+batch_sample[0]+'.png'
                image = cv2.imread(filename)
                image=cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

                # Read label (y)
                y = [x for x in batch_sample[2:]]
                X_train.append(image)
                y_train.append(y)

                filename = 'data\\processed\\brain_window\\'+batch_sample[0]+'.png'
                image = cv2.imread(filename)
                image=cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                X_trains.append(image)

                filename = 'data\\processed\\blood_window\\'+batch_sample[0]+'.png'
                image = cv2.imread(filename)
                image=cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                X_trainss.append(image)
 
            # Make sure they're numpy arrays (as opposed to lists)
            X_train = np.array(X_train)/255.0
            X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)

            X_trains = np.array(X_trains)/255.0
            X_trains = X_trains.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)

            X_trainss = np.array(X_trainss)/255.0
            X_trainss = X_trainss.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)


            # X_train=tf.convert/_to_tensor(X_train)
            y_train = np.array(y_train)
            # y_train=tf.convert_to_tensor(y_train)
            # X_train=X_train.reshape(X_train.shape[0],X_train.shape[1],X_train.shape[2],1)
            # The generator-y part: yield the next training batch            
            yield [X_train,X_trains,X_trainss], y_train

            
train_datagen = generator(samples[0:int(len(samples)*0.8)],batch_size=32)
validation_generator = generator(samples[int(len(samples)*0.8):], batch_size=32)

In [ ]:
# Model Training Cell
# batch=32
# steps_per_epoch = TotalTrainingSamples / TrainingBatchSize
# validation_steps = TotalvalidationSamples / ValidationBatchSize
history=model.fit(
        train_datagen,
        epochs=11, steps_per_epoch=int(len(samples)*0.8/32)
        ,validation_steps=int(0.2 * len(samples)/32)
        ,callbacks=callbacks_list
        ,validation_data=validation_generator)

In [ ]:
from keras.models import load_model
model = load_model('/content/drive/MyDrive/temp/weights-improvement-06-0.633729.hdf5')

In [ ]:
filename = '/content/drive/MyDrive/processed/bone_window/'+'ID_000a050f3'+'.png'              
X_train = []
image = cv2.imread(filename)
image=cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
X_train.append(image)
X_train = np.array(X_train)/255.0
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)

filenames = '/content/drive/MyDrive/processed/brain_window/'+'ID_000a050f3'+'.png'             
X_trains = []
image = cv2.imread(filename)
image=cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
X_trains.append(image)
X_trains = np.array(X_train)/255.0
X_trains = X_trains.reshape(X_trains.shape[0], X_trains.shape[1], X_trains.shape[2], 1)

filenames = '/content/drive/MyDrive/processed/blood_window/'+'ID_000a050f3'+'.png'
X_trainss = []
image = cv2.imread(filenames)
image=cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
X_trainss.append(image)
X_trainss = np.array(X_trainss)/255.0
X_trainss = X_trainss.reshape(X_trains.shape[0], X_trains.shape[1], X_trains.shape[2], 1)

model.predict([X_train,X_trains,X_trainss])

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

UNUSED CELLS BELOW>>>

In [2]:
#Dont Use This Cell 
#Without using Pool and multiprocessing. Doesnt cause Memory overload

data = pd.read_csv("/content/drive/MyDrive/Resources/ct_train_final.csv",index_col=0)
samples=[]  
c=0
bone_input=[]
brain_input=[]
blood_input=[]
label=[]
for row in data.itertuples(index=False):
    samples.append([x for x in row])
    c+=1
    if(c==6000):
      break
for batch_sample in samples:
                # Load image (X)
  filename_bone = '/content/processed/bone_window/'+batch_sample[0]+'.png'
  filename_brain = '/content/processed/brain_window/'+batch_sample[0]+'.png'
  filename_blood = '/content/processed/blood_window/'+batch_sample[0]+'.png'
  image = cv2.imread(filename_bone)
  image=cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  bone_input.append(image)

  # y = [x for x in batch_sample[2:]]
  image = cv2.imread(filename_brain)
  image=cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  brain_input.append(image)

  image = cv2.imread(filename_blood)
  image=cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  blood_input.append(image)

  label.append([x for x in batch_sample[2:]])
bone_input = np.array(bone_input)/255.0
bone_input = bone_input.reshape(bone_input.shape[0], bone_input.shape[1], bone_input.shape[2], 1)

brain_input = np.array(brain_input)/255.0
brain_input = brain_input.reshape(brain_input.shape[0], brain_input.shape[1], brain_input.shape[2], 1)

blood_input = np.array(blood_input)/255.0
blood_input = blood_input.reshape(brain_input.shape[0], brain_input.shape[1], brain_input.shape[2], 1)

# m=tf.convert_to_tensor(m)
label= np.array(label)
# n=tf.convert_to_tensor(n)


In [ ]:
#It is one of two cell to load images faster using multiprocessing if upper cell is executed no need to run
#Part of Pool Cell

data = pd.read_csv("/content/drive/MyDrive/Resources/ct_train_final.csv",index_col=0)
samples=[]  
c=0
bone_input=[]
brain_input=[]
blood_input=[]
label=[]
for row in data.itertuples(index=False):
    samples.append([x for x in row])
    c+=1
    # if(c==5):
    #   break

def load_bone_images(samples):
  filename_bone = '/content/processed/bone_window/'+samples[0]+'.png'
  image = cv2.imread(filename_bone)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  return image

def load_brain_images(samples):
  filename_bone = '/content/processed/brain_window/'+samples[0]+'.png'
  image = cv2.imread(filename_bone)
  image=  cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  return image

def load_blood_images(samples):
  filename_bone = '/content/processed/blood_window/'+samples[0]+'.png'
  image = cv2.imread(filename_bone)
  image=  cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  return image

def load_label(samples):
  # label.append([x for x in batch_sample[2:]])
  return [x for x in samples[2:]];

In [ ]:
#It is the second portion to load images using multiprocessing
#Using Pool causes Memory Overload


bone_input=[]
brain_input=[]
blood_input=[]
label=[]
with Pool(2) as p:
  bone_input.append(p.map(load_bone_images, samples))
  bone_input=bone_input[0]
  bone_input = np.array(bone_input)/255.0
  bone_input = bone_input.reshape(bone_input.shape[0], bone_input.shape[1], bone_input.shape[2], 1)
  p.close()

with Pool(2) as p:
  brain_input.append(p.map(load_brain_images, samples))
  brain_input=brain_input[0]
  brain_input = np.array(brain_input)/255.0
  brain_input = brain_input.reshape(bone_input.shape[0], bone_input.shape[1], bone_input.shape[2], 1)
  p.close()

with Pool(2) as p:
  blood_input.append(p.map(load_brain_images, samples))
  blood_input=blood_input[0]
  blood_input = np.array(blood_input)/255.0
  blood_input = blood_input.reshape(brain_input.shape[0], brain_input.shape[1], brain_input.shape[2], 1)
  p.close()

with Pool(2) as p:
  label.append(p.map(load_label,samples))
  label=label[0]
  label= np.array(label)
  p.close()